In this notebook we will reproduce the results of experiments with merging Named Entities based signal with content similarity measures

# Apply different content similarity measures to the dataset

If you want to reproduce the results of the experiemnt thoroughly strat from this section, calculate standr metrocs and Named Entities based metric first.
If you don't want to recalculate it use 'SGDD-TST_with_metrics.csv' file where all the scores are already present. In thos case you may go to the Section 'Merge Named Entities based metric with regular metrics'

In [6]:
import pandas as pd

In [2]:
df = pd.read_csv("../dataset/SGDD-TST.csv")

In [3]:
df.head()

,INPUT:text_first,INPUT:text_second,OUTPUT:result,CONFIDENCE:result,vote_type,vote_different,vote_some_details_lost,vote_OK,average
0,"4th of March, 4 people going.","On the fourth of March, there will be four peo...",OK,0.9864,12,0,1,2,2.666667
1,I will leave from SFO and want to get back on ...,I'm going to get back from SFO on 7th of march!,OK,0.9999,12,1,0,2,2.333333
2,From Vancouver on Monday next week.,"On Monday next week, I will be in Vancouver.",some_details_lost,0.9999,12,0,2,1,2.333333
3,"I want to continue looking, how about movies d...",i want to keep looking how about movies direct...,some_details_lost,0.9999,12,0,2,1,2.333333
4,"Hi, I need a help, i am interested to some thi...","hi i need some help, can you find some games f...",some_details_lost,0.9929,12,0,2,1,2.333333


Run prepare_metric_inferenceto have all necessary files ready

In [ ]:
!chmod 755 ./prepare_metric_inference.sh
!./prepare_metric_inference.sh

In [11]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [29]:
from all_metrics import score

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [30]:
score(df['INPUT:text_first'].tolist(),
     df['INPUT:text_second'].tolist(),
     df['average'].tolist(),
     <SAVE_NAME>,#'SGDD-TST_with_metrics.csv' already has all metrics calculated, you may create a new file if you want to reproduce claculations
     calculate_long = False) # set calculate_long to True if you want to get scores from meteor

w2v/fasttext
w2v


  0%|          | 0/10287 [00:00<?, ?it/s]

/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


No exception raised!


In [8]:
df.head()

,text1,text2,human,chrf,bleu,rouge1,rouge2,rouge3,rougeL,Elron/bleurt-large-512,bertscore/roberta-large,bertscore/bert-base-multilingual-cased,bertscore/microsoft/deberta-xlarge-mnli,fasttext_cossim,w2v_cossim
0,"4th of March, 4 people going.","On the fourth of March, there will be four peo...",2.666667,0.453440,0.349133,0.352941,0.133333,0.000000,0.352941,0.375168,0.938417,0.826329,0.831641,0.756650,0.726442
1,I will leave from SFO and want to get back on ...,I'm going to get back from SFO on 7th of march!,2.333333,0.428982,0.546568,0.769231,0.500000,0.272727,0.615385,0.318186,0.947726,0.863049,0.851195,0.978753,0.871557
2,From Vancouver on Monday next week.,"On Monday next week, I will be in Vancouver.",2.333333,0.613840,0.596271,0.666667,0.461538,0.363636,0.533333,0.145527,0.937152,0.816597,0.804163,0.818123,0.901341
3,"I want to continue looking, how about movies d...",i want to keep looking how about movies direct...,2.333333,0.554911,0.676329,0.846154,0.666667,0.454545,0.846154,0.378284,0.928749,0.825081,0.915673,0.926830,0.899193
4,"Hi, I need a help, i am interested to some thi...","hi i need some help, can you find some games f...",2.333333,0.235932,0.179991,0.578947,0.333333,0.176471,0.526316,-0.242777,0.927132,0.822242,0.814345,0.889586,0.852515


# Apply Named Entities based measure

In [11]:
from ner_utils import normalize_ner, ner_processor, unfold_list, get_ner_lists_smart_intersection
from tqdm.auto import tqdm

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [12]:
import spacy
import re

nlp = spacy.load('en_core_web_md')

In [13]:
sent1 = df['text1'].tolist()
sent2 = df['text2'].tolist()

sent1_ner, sent1_nercount, sent1_noner_count = unfold_list([ner_processor(t, nlp) for t in tqdm(sent1)])
sent2_ner, sent2_nercount, sent2_noner_count = unfold_list([ner_processor(t, nlp) for t in tqdm(sent2)])

  0%|          | 0/10287 [00:00<?, ?it/s]

  0%|          | 0/10287 [00:00<?, ?it/s]

In [14]:
metric = [get_ner_lists_smart_intersection(n1, n2, normalize_ner, substring_search = False) for n1,n2 in zip(tqdm(sent1_ner), sent2_ner)]
df['scipy_ner'] = metric

  0%|          | 0/10287 [00:00<?, ?it/s]

In [15]:
total_ner_count = [l1+l2 for l1,l2 in zip(sent1_nercount,sent2_nercount)]
total_noner_count = [l1+l2 for l1,l2 in zip(sent1_noner_count,sent2_noner_count)]

In [16]:
df['scipy_proprotion'] = [ner/(ner+noner) for ner,noner in zip(total_ner_count, total_noner_count)]

In [17]:
df.head()

,text1,text2,human,chrf,bleu,rouge1,rouge2,rouge3,rougeL,Elron/bleurt-large-512,bertscore/roberta-large,bertscore/bert-base-multilingual-cased,bertscore/microsoft/deberta-xlarge-mnli,fasttext_cossim,w2v_cossim,scipy_ner,scipy_proprotion
0,"4th of March, 4 people going.","On the fourth of March, there will be four peo...",2.666667,0.453440,0.349133,0.352941,0.133333,0.000000,0.352941,0.375168,0.938417,0.826329,0.831641,0.756650,0.726442,0.5,0.529412
1,I will leave from SFO and want to get back on ...,I'm going to get back from SFO on 7th of march!,2.333333,0.428982,0.546568,0.769231,0.500000,0.272727,0.615385,0.318186,0.947726,0.863049,0.851195,0.978753,0.871557,0.5,0.230769
2,From Vancouver on Monday next week.,"On Monday next week, I will be in Vancouver.",2.333333,0.613840,0.596271,0.666667,0.461538,0.363636,0.533333,0.145527,0.937152,0.816597,0.804163,0.818123,0.901341,1.0,0.600000
3,"I want to continue looking, how about movies d...",i want to keep looking how about movies direct...,2.333333,0.554911,0.676329,0.846154,0.666667,0.454545,0.846154,0.378284,0.928749,0.825081,0.915673,0.926830,0.899193,0.0,0.115385
4,"Hi, I need a help, i am interested to some thi...","hi i need some help, can you find some games f...",2.333333,0.235932,0.179991,0.578947,0.333333,0.176471,0.526316,-0.242777,0.927132,0.822242,0.814345,0.889586,0.852515,0.0,0.000000


# Merge Named Entities based metric with regular metrics

In [1]:
import pandas as pd
import re
df = pd.read_csv("SGDD-TST_with_metrics.csv")

In [2]:
from scipy.stats import spearmanr

In [3]:
%%capture --no-stdout

metric_list = []
corr_list = []
corr_with_total_merge = []
ner_columns = ['scipy_proprotion','scipy_ner']

ner_dict_list = {'weight_by_scipy_ner':[]}

for m in df.columns:
    if m not in ['text1','text2','human','scipy_ner','scipy_proprotion']:
        print(m)
        
        dfc = df[['human',m]+ner_columns]
        dfc.dropna(inplace = True)
        if len(dfc) < len(df):
            diff = len(df) - len(dfc)
            print(f"Dropped {diff} null-elements for metric <{m}>")

        human = dfc['human'].tolist()
        metric = dfc[m].tolist()
   
        corr_orig = spearmanr(metric, human).correlation
        corr_list.append(corr_orig)
        metric_list.append(m)
                
        dfc["merge_by_token_weight_total"] = dfc.apply(lambda x: x[m]*(1-x['scipy_proprotion']) + 
                                                                           x['scipy_ner']*(x['scipy_proprotion']), axis = 1)

        metric_weighted = dfc["merge_by_token_weight_total"].tolist()

        corr_weighted = spearmanr(metric_weighted, human).correlation
        ner_dict_list['weight_by_scipy_ner'].append(corr_weighted)
        print("-"*100)
                
df_ner_weighted = pd.DataFrame({'metric':metric_list, "corr":corr_list}) 
for ner_metric, correl_weighted in ner_dict_list.items():
    df_ner_weighted[ner_metric] = correl_weighted

df_ner_weighted = df_ner_weighted.sort_values(by = 'corr', ascending = False)

chrf
----------------------------------------------------------------------------------------------------
bleu
----------------------------------------------------------------------------------------------------
rouge1
----------------------------------------------------------------------------------------------------
rouge2
----------------------------------------------------------------------------------------------------
rouge3
----------------------------------------------------------------------------------------------------
rougeL
----------------------------------------------------------------------------------------------------
Elron/bleurt-large-512
----------------------------------------------------------------------------------------------------
bertscore/roberta-large
----------------------------------------------------------------------------------------------------
bertscore/bert-base-multilingual-cased
--------------------------------------------------------------------

In [4]:
df_ner_weighted

,metric,corr,weight_by_scipy_ner
6,Elron/bleurt-large-512,0.561901,0.558323
9,bertscore/microsoft/deberta-xlarge-mnli,0.465590,0.451859
7,bertscore/roberta-large,0.397370,0.367713
1,bleu,0.348286,0.375503
2,rouge1,0.292041,0.358414
8,bertscore/bert-base-multilingual-cased,0.284003,0.356662
5,rougeL,0.271009,0.349383
0,chrf,0.268488,0.301935
11,w2v_cossim,0.216380,0.329118
10,fasttext_cossim,0.215449,0.323822


## Check whether the correlation increase is significant with Williams test

In [5]:
import numpy as np
from scipy import stats

def williams_test(r12, r13, r23, n):
    """The Williams test (Evan J. Williams. 1959. Regression Analysis, volume 14. Wiley, New York, USA)
    
    A test of whether the population correlation r12 equals the population correlation r13.
    Significant: p < 0.05
    
    Arguments:
        r12 (float): correlation between x1, x2
        r13 (float): correlation between x1, x3
        r23 (float): correlation between x2, x3
        n (int): size of the population
        
    Returns:
        t (float): Williams test result
        p (float): p-value of t-dist
    """
#     print("r12 {}, r13 {} r23 {}".format(r12, r13, r23))
    if r12 < r13:
#         print('r12 should be larger than r13')
        return 0,0
    elif n <= 3:
#         print('n should be larger than 3')
        return 0,0
    else:
        K = 1 - r12**2 - r13**2 - r23**2 + 2*r12*r13*r23
        denominator = np.sqrt(2*K*(n-1)/(n-3) + (((r12+r13)**2)/4)*((1-r23)**3))
        numerator = (r12-r13) * np.sqrt((n-1)*(1+r23))
        t = numerator / denominator
        p = 1 - stats.t.cdf(t, df=n-3) # changed to n-3 on 30/11/14
        return t, p
    
def decide_if_increase_significant(strong_metric_corr, close2strong_metric_corr, strong_weighted_name, original_strong_name, verbose = False):
 
    r12 = strong_metric_corr 
    r13 = close2strong_metric_corr
    
    assert r12>r13
    
    pair = '_'.join(sorted([strong_weighted_name, original_strong_name]))
    r23 = ner_corr_dict[pair]
    
    n = len(df)

    res = williams_test(r12, r13, r23, n)
    if res:
        pvalue = res[1]
        if pvalue < 0.05:
            if verbose == True: print("Significant Difference! pvalue = {}, stronger_corr = {}, weaker_corr = {}".format(pvalue, strong_metric_corr, close2strong_metric_corr))
            return True
        elif pvalue >= 0.05:
            if verbose == True: print("Not Significant Difference! pvalue = {}, stronger_corr = {}, weaker_corr = {}".format(pvalue, strong_metric_corr, close2strong_metric_corr))
            return False

In [6]:
ner_corr_dict = {}
for strong_met in df_ner_weighted['metric']:
    pair = '_'.join(sorted(['scipy_ner', strong_met]))
    if pair not in ner_corr_dict:

        df_curr = df[['scipy_ner',strong_met ]]
        df_curr.dropna(inplace = True)

        m1 = df_curr['scipy_ner']
        m2 = df_curr[strong_met]
        cr = spearmanr(m1, m2).correlation
        ner_corr_dict[pair] = cr

/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [17]:
VERBOSE = True
signif_dict = {'signif_scipy_ner':[]}

df_rep_weighted_moreinfo = df_ner_weighted.copy()

for i,el in df_ner_weighted.iterrows():
    if VERBOSE == True: print(el['metric'])
    close2strong_metric_corr = el['corr']
    original_strong_name = el['metric']
    for weight_metric in df_rep_weighted_moreinfo.columns[2:]:
        strong_weighted_name = re.sub('weight_by_','',weight_metric)
        if el[weight_metric] > el['corr']:
            if VERBOSE == True: print(weight_metric)
            strong_metric_corr = el[weight_metric]
            is_significant = decide_if_increase_significant(strong_metric_corr, close2strong_metric_corr, strong_weighted_name, original_strong_name, verbose = VERBOSE)
        else: 
#             is_significant = decide_if_increase_significant(close2strong_metric_corr, strong_metric_corr,original_strong_name, strong_weighted_name, verbose = VERBOSE)
#             is_significant = not is_significant
             is_significant = False
            
        signif_dict[f'signif_{strong_weighted_name}'].append(is_significant)
    if VERBOSE == True: print("="*100)
#     break
    
for ner_metric_sign, signif_list in signif_dict.items():
    df_rep_weighted_moreinfo[ner_metric_sign] = signif_list

Elron/bleurt-large-512
bertscore/microsoft/deberta-xlarge-mnli
bertscore/roberta-large
bleu
weight_by_scipy_ner
Significant Difference! pvalue = 0.005592402489985382, stronger_corr = 0.3755030648225277, weaker_corr = 0.34828591349277077
rouge1
weight_by_scipy_ner
Significant Difference! pvalue = 1.1451101178394651e-09, stronger_corr = 0.3584137870345997, weaker_corr = 0.29204129744277274
bertscore/bert-base-multilingual-cased
weight_by_scipy_ner
Significant Difference! pvalue = 8.033329557122215e-11, stronger_corr = 0.35666208621754397, weaker_corr = 0.284003225549297
rougeL
weight_by_scipy_ner
Significant Difference! pvalue = 3.13915560212763e-12, stronger_corr = 0.3493834509993796, weaker_corr = 0.2710088262057129
chrf
weight_by_scipy_ner
Significant Difference! pvalue = 0.001236078184662981, stronger_corr = 0.30193533323979793, weaker_corr = 0.26848773502900175
w2v_cossim
weight_by_scipy_ner
Significant Difference! pvalue = 0.0, stronger_corr = 0.3291179475792798, weaker_corr = 0.21

In [18]:
def hlght(x):
    if x['signif_scipy_ner'] == True:
        return ['background-color: lightgreen']*len(x)
    else:
        return ['background-color: white']*len(x)

In [19]:
df_rep_weighted_moreinfo.style.apply(hlght, axis=1)

,metric,corr,weight_by_scipy_ner,signif_scipy_ner
6,Elron/bleurt-large-512,0.561901,0.558323,False
9,bertscore/microsoft/deberta-xlarge-mnli,0.465590,0.451859,False
7,bertscore/roberta-large,0.397370,0.367713,False
1,bleu,0.348286,0.375503,True
2,rouge1,0.292041,0.358414,True
8,bertscore/bert-base-multilingual-cased,0.284003,0.356662,True
5,rougeL,0.271009,0.349383,True
0,chrf,0.268488,0.301935,True
11,w2v_cossim,0.216380,0.329118,True
10,fasttext_cossim,0.215449,0.323822,True


In [20]:
df_rep_weighted_moreinfo['corr'] = df_rep_weighted_moreinfo['corr'].apply(lambda x: round(x,2))
df_rep_weighted_moreinfo['weight_by_scipy_ner'] = df_rep_weighted_moreinfo['weight_by_scipy_ner'].apply(lambda x: round(x,2))

In [22]:
df_rep_weighted_moreinfo

,metric,corr,weight_by_scipy_ner,signif_scipy_ner
6,Elron/bleurt-large-512,0.56,0.56,False
9,bertscore/microsoft/deberta-xlarge-mnli,0.47,0.45,False
7,bertscore/roberta-large,0.40,0.37,False
1,bleu,0.35,0.38,True
2,rouge1,0.29,0.36,True
8,bertscore/bert-base-multilingual-cased,0.28,0.36,True
5,rougeL,0.27,0.35,True
0,chrf,0.27,0.30,True
11,w2v_cossim,0.22,0.33,True
10,fasttext_cossim,0.22,0.32,True


In [23]:
df_rep_weighted_moreinfo.to_csv("t.tsv", index = None, sep = '\t')

# Conclusion

We have used numerous content similarity measures as is, and we used a Named Entities-based measure as an auxiliary signal. In the table above we have shown that for most standard measures (like BLEU or rouge) and even for some more sophisticated ones (e.g. based on word2vec) this approach yields siginifcant improvement in correlation with human judgements.